In [1]:
import os
import pandas as pd
import gradio as gr

file_word = pd.read_excel("doc/speaker_wordlist.xls", sheet_name="Word_filename")
fw, wf = {}, {}
for _, row in file_word.iterrows():
    code = row["FILE NAME"]
    word = row["WORD"]
    wf[word] = code
    fw[code] = word

/home/myid/zis35724/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(wf["equilibrium"])

B3_UW69


In [2]:
noisered_path = "/data/zaki/UASpeech/audio/noisereduce"
participants = {}

for p in os.listdir(noisered_path):
    files = []
    word_blocks = {}
    for f in os.listdir(f"{noisered_path}/{p}"):
        if not f.endswith('.wav') or f.startswith("._"):
            continue
            
        split = f.replace(f"{p}_", "").replace(".wav", "").split("_")
        block = split[0]
        code = split[1]
        mic = split[2]

        code = f"{block}_{code}" if code.startswith("U") else code
        word = fw[code]
        
        if word_blocks.get(word) is None:
            word_blocks[word] = { block : [mic] }
        else:
            if word_blocks[word].get(block) is None:
                word_blocks[word][block] = [mic]
            else:
                word_blocks[word][block].append(mic)
    participants[p] = word_blocks

In [3]:
def play_audio(participant, word, block, mic):
    if any([x is None for x in [participant, word, block, mic]]):
        return None
    code = wf[word]
    file_path = f"{noisered_path}/{participant}/{participant}_{code}_{mic}.wav" if code.startswith("B") else \
        f"{noisered_path}/{participant}/{participant}_{block}_{code}_{mic}.wav"
    print(file_path)
    return gr.update(value=file_path)

def update_words(p):
    if p is None:
        return None
    choices = sorted(list(participants[p].keys()))
    return gr.update(choices=choices, value=choices[0])

def update_blocks(p, w):
    if any([p is None, w is None]):
        return None
    choices = sorted(list(participants[p][w].keys()))
    print(list(participants[p][w].keys()))
    return gr.update(choices=choices, value=choices[0])

def update_mics(p, w, b):
    if any([x is None for x in [p, w, b]]):
        return None
    choices = sorted(participants[p][w][b])
    print(participants[p][w][b])
    return gr.update(choices=choices, value=choices[0])
    

first_participant = "CF02"
word_choices = sorted(list(participants[first_participant].keys()))

first_word = word_choices[0]
block_choices = sorted(list(participants[first_participant][first_word].keys()))

first_block = list(participants[first_participant][first_word].keys())[0]

mic_choices = sorted(participants[first_participant][first_word][first_block])
first_mic = mic_choices[0]

with gr.Blocks() as app:
    with gr.Row():
        participant_dropdown1 = gr.Dropdown(choices=sorted(list(participants.keys())), label="Select a Participant")
        participant_dropdown2 = gr.Dropdown(choices=sorted(list(participants.keys())), label="Select a Participant")

    with gr.Row():
        word_dropdown1 = gr.Dropdown(choices=word_choices, label="Select word")
        word_dropdown2 = gr.Dropdown(choices=word_choices, label="Select word")

    with gr.Row():
        block_dropdown1 = gr.Dropdown(choices=block_choices, label="Select block")
        block_dropdown2 = gr.Dropdown(choices=block_choices, label="Select block")

    with gr.Row():
        mic_dropdown1 = gr.Dropdown(choices=mic_choices, label="Select mic")
        mic_dropdown2 = gr.Dropdown(choices=mic_choices, label="Select mic")

    with gr.Row():
        audio_output1 = gr.Audio()
        audio_output2 = gr.Audio()

    participant_dropdown1.change(fn=update_words, inputs=participant_dropdown1, outputs=word_dropdown1)
    word_dropdown1.change(fn=update_blocks, inputs=[participant_dropdown1, word_dropdown1], outputs=block_dropdown1)
    block_dropdown1.change(fn=update_mics, inputs=[participant_dropdown1, word_dropdown1, block_dropdown1], outputs=mic_dropdown1)
    mic_dropdown1.change(fn=play_audio, inputs=[participant_dropdown1, word_dropdown1, block_dropdown1, mic_dropdown1], outputs=audio_output1)

    participant_dropdown2.change(fn=update_words, inputs=participant_dropdown2, outputs=word_dropdown2)
    word_dropdown2.change(fn=update_blocks, inputs=[participant_dropdown2, word_dropdown2], outputs=block_dropdown2)
    block_dropdown2.change(fn=update_mics, inputs=[participant_dropdown2, word_dropdown2, block_dropdown2], outputs=mic_dropdown2)
    mic_dropdown2.change(fn=play_audio, inputs=[participant_dropdown2, word_dropdown2, block_dropdown2, mic_dropdown2], outputs=audio_output2)

# Launch the interface. Use `inbrowser=True` to open it directly in the notebook.
app.launch(inbrowser=True, share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://7840c350021fd85b99.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


['B2', 'B1', 'B3']
['M8', 'M2', 'M4', 'M7', 'M5', 'M3', 'M6']
/data/zaki/UASpeech/audio/noisereduce/F04/F04_B1_LA_M2.wav
['B1', 'B3', 'B2']
['M5', 'M3', 'M8', 'M2', 'M4', 'M6', 'M7']
/data/zaki/UASpeech/audio/noisereduce/F04/F04_B3_CW30_M4.wav
['B2', 'B3', 'B1']
['M7', 'M6', 'M3', 'M5', 'M8']
/data/zaki/UASpeech/audio/noisereduce/M04/M04_B1_LA_M3.wav
['B3', 'B2', 'B1']
/data/zaki/UASpeech/audio/noisereduce/M04/M04_B1_CW24_M5.wav
/data/zaki/UASpeech/audio/noisereduce/M04/M04_B1_CW24_M3.wav
['B2', 'B3', 'B1']
['B1']
/data/zaki/UASpeech/audio/noisereduce/M05/M05_B1_UW4_M6.wav
/data/zaki/UASpeech/audio/noisereduce/M05/M05_B1_UW4_M7.wav


In [5]:
min((0,1,2))

0